### Import Necessary Module and Set Path 

In [1]:
import sqlite3 as lite
import os
import sys
import collections
sys.path.insert(0, '../../scripts/')
jupyter_directory = os.getcwd()

In [2]:
import analysis as an

In [24]:
#rle.generate_input('./default_input_file.xml', './random_lifetime_extended_french_lwrs.xml', 720, 'France')
# if you uncomment and run this it will replace the existing input file,
# which means you have to input the astrid deployment manually.

In [36]:
### RUN ALL THE DISTRIBUTIONS OF SHAPES AND SIZES
# I suggest you use the input files already in the repo
#! cyclus -o pois_10.sqlite pois_10.xml
#! cyclus -o pois_15.sqlite pois_15.xml
#! cyclus -o norm_10_3.sqlite norm_10_3.xml
#! cyclus -o norm_15_3.sqlite norm_15_3.xml
#! cyclus -o unif_0_25.sqlite unif_0_25.xml
#! cyclus -o unif_10_25.sqlite unif_10_25.xml

              :                                                               
          .CL:CC CC             _Q     _Q  _Q_Q    _Q    _Q              _Q   
        CC;CCCCCCCC:C;         /_\)   /_\)/_/\\)  /_\)  /_\)            /_\)  
        CCCCCCCCCCCCCl       __O|/O___O|/O_OO|/O__O|/O__O|/O____________O|/O__
     CCCCCCf     iCCCLCC     /////////////////////////////////////////////////
     iCCCt  ;;;;;.  CCCC                                                      
    CCCC  ;;;;;;;;;. CClL.                          c                         
   CCCC ,;;       ;;: CCCC  ;                   : CCCCi                       
    CCC ;;         ;;  CC   ;;:                CCC`   `C;                     
  lCCC ;;              CCCC  ;;;:             :CC .;;. C;   ;    :   ;  :;;   
  CCCC ;.              CCCC    ;;;,           CC ;    ; Ci  ;    :   ;  :  ;  
   iCC :;               CC       ;;;,        ;C ;       CC  ;    :   ; .      
  CCCi ;;               CCC        ;;;.      .C ;   

Agent entering now will be registered: CATTENOM-1
Agent entering now will be registered: CATTENOM-2
Agent entering now will be registered: BELLEVILLE-1
Agent entering now will be registered: NOGENT-1
Agent entering now will be registered: CHINON B-4
Agent entering now will be registered: BELLEVILLE-2
Agent entering now will be registered: NOGENT-2
Agent entering now will be registered: CATTENOM-3
Agent entering now will be registered: PENLY-1
Agent entering now will be registered: GOLFECH-1
Agent exiting now will be unregistered: ST. LAURENT A-1
Agent exiting now will be unregistered: ST. LAURENT A-2
Agent exiting now will be unregistered: CHINON A-3
Agent entering now will be registered: CATTENOM-4
Agent exiting now will be unregistered: CHOOZ-A
Agent entering now will be registered: PENLY-2
Agent entering now will be registered: GOLFECH-2
Agent exiting now will be unregistered: BUGEY-1
Agent entering now will be registered: MELOX
Agent entering now will be registered: CHOOZ B-1
Agent

Agent entering now will be registered: astrid
Agent exiting now will be unregistered: BLAYAIS-4
Coin0506I Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Clp0000I Optimal - objective value 650
Coin0511I After Postsolve, objective 650, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 650 - 0 iterations time 0.002, Presolve 0.00
 To enjoy the full performance of Cbc, use the CbcSolver interface.
Clp0000I Optimal - objective value 650
Cbc0045I Heuristic rounding took 0 seconds (no good)
Clp0000I Optimal - objective value 650
Cbc0046I Root node pass 1, 1 rows, 0 total tight cuts  -  objective 650
Clp0006I 0  Obj 650 Primal inf 0.91666657 (1)
Clp0006I 1  Obj 1200
Clp0000I Optimal - objective value 1200
Clp0000I Optimal - objective value 1200
Cbc0013I At root node, 0 cuts changed objective from 650 to 1200 in 1 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 1 column cuts (1 active)  in 0.000 seconds - new frequency is -100
Cbc0014I 

Agent entering now will be registered: astrid
Agent exiting now will be unregistered: GRAVELINES-5
Coin0506I Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Clp0000I Optimal - objective value 675
Coin0511I After Postsolve, objective 675, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 675 - 0 iterations time 0.002, Presolve 0.00
 To enjoy the full performance of Cbc, use the CbcSolver interface.
Clp0000I Optimal - objective value 675
Cbc0045I Heuristic rounding took 0 seconds (no good)
Clp0000I Optimal - objective value 675
Cbc0046I Root node pass 1, 1 rows, 0 total tight cuts  -  objective 675
Clp0006I 0  Obj 675 Primal inf 0.8749999 (1)
Clp0006I 1  Obj 1200
Clp0000I Optimal - objective value 1200
Clp0000I Optimal - objective value 1200
Cbc0013I At root node, 0 cuts changed objective from 675 to 1200 in 1 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 1 column cuts (1 active)  in 0.000 seconds - new frequency is -100
Cbc0014

Agent entering now will be registered: astrid
Agent entering now will be registered: astrid
Agent exiting now will be unregistered: ST. LAURENT B-1
Agent exiting now will be unregistered: LA_HAGUE
Agent exiting now will be unregistered: MELOX
Coin0506I Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Clp0000I Optimal - objective value 420
Coin0511I After Postsolve, objective 420, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 420 - 0 iterations time 0.002, Presolve 0.00
 To enjoy the full performance of Cbc, use the CbcSolver interface.
Clp0000I Optimal - objective value 420
Cbc0045I Heuristic rounding took 1e-06 seconds (no good)
Clp0000I Optimal - objective value 420
Cbc0046I Root node pass 1, 1 rows, 0 total tight cuts  -  objective 420
Clp0006I 0  Obj 420 Primal inf 0.2999999 (1)
Clp0006I 1  Obj 600
Clp0000I Optimal - objective value 600
Clp0000I Optimal - objective value 600
Cbc0013I At root node, 0 cuts changed objective from 420 to 600 in 1 passes


...

Agent exiting now will be unregistered: astrid
Agent exiting now will be unregistered: astrid
Coin0506I Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Clp0000I Optimal - objective value 1200
Coin0511I After Postsolve, objective 1200, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 1200 - 0 iterations time 0.002, Presolve 0.00
 To enjoy the full performance of Cbc, use the CbcSolver interface.
Clp0000I Optimal - objective value 1200
Cbc0045I Heuristic rounding took 0 seconds (no good)
Clp0000I Optimal - objective value 1200
Cbc0004I Integer solution of 1200 found after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 1200, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Clp0000I Optimal - objective value 1200
Clp0000I Optimal - objective value 1200
Agent entering now will be registered: astrid
Agent entering now will be registered: astrid
Agent exiting now will be u

Agent exiting now will be unregistered: astrid
Agent exiting now will be unregistered: astrid
Coin0506I Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Clp0000I Optimal - objective value 1200
Coin0511I After Postsolve, objective 1200, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 1200 - 0 iterations time 0.002, Presolve 0.00
 To enjoy the full performance of Cbc, use the CbcSolver interface.
Clp0000I Optimal - objective value 1200
Cbc0045I Heuristic rounding took 1e-06 seconds (no good)
Clp0000I Optimal - objective value 1200
Cbc0004I Integer solution of 1200 found after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 1200, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Clp0000I Optimal - objective value 1200
Clp0000I Optimal - objective value 1200
Agent exiting now will be unregistered: astrid
Agent exiting now will be unregistered: astrid
Agent entering now wi

Clp0000I Optimal - objective value 1800
Cbc0004I Integer solution of 1800 found after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 1800, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Clp0000I Optimal - objective value 1800
Clp0000I Optimal - objective value 1800
Agent exiting now will be unregistered: astrid
Agent exiting now will be unregistered: astrid
Agent exiting now will be unregistered: astrid
Agent exiting now will be unregistered: astrid
Agent entering now will be registered: astrid
Agent entering now will be registered: astrid
Agent entering now will be registered: astrid
Coin0506I Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Clp0000I Optimal - objective value 2400
Coin0511I After Postsolve, objective 2400, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 2400 - 0 iterations time 0.002, Presolve 0.00
 To enjoy the full performance of Cbc, use the C

Agent entering now will be registered: astrid
Agent entering now will be registered: astrid
Agent exiting now will be unregistered: astrid
Agent exiting now will be unregistered: astrid
Coin0506I Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Clp0000I Optimal - objective value 1200
Coin0511I After Postsolve, objective 1200, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 1200 - 0 iterations time 0.002, Presolve 0.00
 To enjoy the full performance of Cbc, use the CbcSolver interface.
Clp0000I Optimal - objective value 1200
Cbc0045I Heuristic rounding took 1e-06 seconds (no good)
Clp0000I Optimal - objective value 1200
Cbc0004I Integer solution of 1200 found after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 1200, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Clp0000I Optimal - objective value 1200
Clp0000I Optimal - objective value 1200
Agent entering now will

Agent entering now will be registered: astrid
Agent entering now will be registered: astrid
Agent exiting now will be unregistered: astrid
Agent exiting now will be unregistered: astrid
Coin0506I Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Clp0000I Optimal - objective value 1200
Coin0511I After Postsolve, objective 1200, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 1200 - 0 iterations time 0.002, Presolve 0.00
 To enjoy the full performance of Cbc, use the CbcSolver interface.
Clp0000I Optimal - objective value 1200
Cbc0045I Heuristic rounding took 0 seconds (no good)
Clp0000I Optimal - objective value 1200
Cbc0004I Integer solution of 1200 found after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 1200, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Clp0000I Optimal - objective value 1200
Clp0000I Optimal - objective value 1200
Agent entering now will be 

In [ ]:
# just france transition

list_of_files = ['pois_10.sqlite', 'pois_15.sqlite',
                 'norm_15_3.sqlite', 'unif_0_25.sqlite',
                 'unif_10_25.sqlite', 'norm_10_6.sqlite']
for file in list_of_files:
    cur = an.get_cursor(file)

    parentid = cur.execute('SELECT agentid FROM agententry WHERE '
                           'Prototype LIKE "%france%" '
                           'AND Kind = "Inst"').fetchone()
    entry_exit = cur.execute('SELECT max(value), timeseriespower.agentid, '
                             'parentid, entertime, prototype, entertime + lifetime'
                             ' FROM agententry '
                             'INNER JOIN timeseriespower '
                             'ON agententry.agentid = timeseriespower.agentid '
                             'WHERE parentid = %i '
                             'GROUP BY timeseriespower.agentid' %parentid[0]).fetchall()

    init_year, init_month, duration, timestep = an.get_timesteps(cur)
    dictionary = {}
    lwr_cap = 0
    astrid_cap = 0
    dictionary['lwr'] = []
    dictionary['astrid'] = []
    for t in timestep:
        for agent in entry_exit:
            if (agent['entertime'] == t) and agent['prototype'] == 'astrid':
                astrid_cap += agent['max(value)'] * 0.001
            if (agent['entertime'] == t) and agent['prototype'] != 'astrid':
                lwr_cap += agent['max(value)'] * 0.001

            if (agent['entertime + lifetime'] == t) and agent['prototype'] == 'astrid':
                astrid_cap -= agent['max(value)'] * 0.001
            if (agent['entertime + lifetime'] == t) and agent['prototype'] != 'astrid':
                lwr_cap -= agent['max(value)'] * 0.001
        dictionary['lwr'].append(lwr_cap)
        dictionary['astrid'].append(astrid_cap)
    an.stacked_bar_chart(dictionary, timestep,
                      'Years', 'Net_Capacity [GWe]',
                      'Net Capacity vs Time',
                      file + '_power_plot.png', init_year)


astrid has no values


/home/dkadkf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1710: MatplotlibDeprecationWarning: The *left* kwarg to `bar` is deprecated use *x* instead. Support for *left* will be removed in Matplotlib 3.0
  return func(ax, *args, **kwargs)


astrid has no values
astrid has no values
astrid has no values
astrid has no values


In [5]:
# generate number of astrid deployed per timestep for all shapes and sizes

for file in list_of_files:
    cur = an.get_cursor('plus960.sqlite')
    parentid = cur.execute('SELECT agentid FROM agententry WHERE '
                           'Prototype LIKE "%france%" '
                           'AND Kind = "Inst"').fetchone()
    entry_exit = cur.execute('SELECT max(value), timeseriespower.agentid, '
                             'parentid, entertime, prototype, entertime + lifetime'
                             ' FROM agententry '
                             'INNER JOIN timeseriespower '
                             'ON agententry.agentid = timeseriespower.agentid '
                             'WHERE parentid = %i '
                             'GROUP BY timeseriespower.agentid' %parentid[0]).fetchall()

    init_year, init_month, duration, timestep = an.get_timesteps(cur)
    dictionary = {}
    cum_dict = {}
    num_dict = {}
    cum_dict['cum'] = []
    num_dict['num'] = []
    astrid_cum = 0
    for t in timestep:
        astrid_num = 0
        for agent in entry_exit:
            if (agent['entertime'] == t) and agent['prototype'] == 'astrid':
                astrid_num += 1
                astrid_cum += 1

            if (agent['entertime + lifetime'] == t) and agent['prototype'] == 'astrid':
                astrid_cum -= 1

        cum_dict['cum'].append(astrid_cum)
        num_dict['num'].append(astrid_num)

    print(([x-110 for x in cum_dict['cum']].index(0)))
    an.double_axis_bar_line_plot(num_dict, cum_dict, timestep,
                                 'Years', 'Number of SFRs Deployed', ' Number of SFRs Operational',
                                 'Build number and cumulative ASTRID Number vs Time', 'sfr_deploy.png',
                                 init_year)

1290
1290
1290
1290
1290
1290
